# Preparation

<b>Libraries</b>

In [1]:
# bs4
import requests
from bs4 import BeautifulSoup

# selenium
from selenium import webdriver
from selenium.webdriver import ChromeOptions
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import Select
from selenium.webdriver.chrome.webdriver import WebDriver
from selenium.webdriver.remote.webelement import WebElement

# data structures
import numpy as np
import pandas as pd

# database
import sqlite3

# others
import sys, os, re, datetime, time

<b>UDF</b>

In [2]:
# login to redfin
def login_to_redfin(browser: WebDriver, email: str, password: str) -> None:
    time.sleep(5)
    ## begin logging
    browser.find_element(By.XPATH, "//span[text()='Join / Sign in']/..").click()
    ## email
    time.sleep(1)
    browser.find_element(By.XPATH, "//input[@name='emailInput']").send_keys(email)
    browser.find_element(By.XPATH, "//span[text()='Continue with Email']/..").click()
    ## password
    time.sleep(1)
    browser.find_element(By.XPATH, "//input[@name='passwordInput']").send_keys(password)
    browser.find_element(By.XPATH, "//span[text()='Continue with Email']/..").click()

    return None

In [3]:
# get links
def get_cities_links(browser: WebDriver) -> tuple[list, list]:
    time.sleep(1)
    ## get list of cities
    cities = browser.find_elements(By.XPATH, f"//span[text()='Search for homes by city']/following-sibling::ul/child::li")
    ## get link for each city
    names = [city.text.strip() for city in cities]
    links = [city.find_element(By.XPATH, ".//child::a").get_attribute('href') for city in cities]

    return names, links

In [4]:
# get link to download the csv file
def get_csv_link(browser: WebDriver) -> str:
    link =  browser.find_element(By.XPATH, "//a[text()='(Download All)']").get_attribute('href')

    return link

# Selenium

<b>Scrapping</b>

In [ ]:
# header: user-agent
user_agent = 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/60.0.3112.50 Safari/537.36'
# options
chrome_options = ChromeOptions()
chrome_options.add_argument(f'user-agent={user_agent}')
# browser
url_chicago = 'https://www.redfin.com/city/29470/IL/Chicago'
browser = webdriver.Chrome(options=chrome_options)
browser.get(url_chicago)
browser.implicitly_wait(10)

In [ ]:
email = 'john.lukestein@gmail.com'
password = 'redfin.0504'
login_to_redfin(browser, email, password)

<b>Draft</b>

In [5]:
# header: user-agent
user_agent = 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/60.0.3112.50 Safari/537.36'
# options
chrome_options = ChromeOptions()
chrome_options.add_argument(f'user-agent={user_agent}')
# browser
url_redfin = 'https://www.redfin.com/'
browser = webdriver.Chrome(options=chrome_options)
browser.get(url_redfin)
time.sleep(5)

In [6]:
# redfin logging
email = 'john.lukestein@gmail.com'
password = 'redfin.0504'
login_to_redfin(browser, email, password)

Forget to get the name of the city

In [51]:
# store links downloading csv files in a txt file
cities_links = get_cities_links(browser)
for link in cities_links:
    browser.get(link)
    time.sleep(1)
    try:
        link = get_csv_link(browser)
        with open('../resource/data/csv_links.txt', 'a') as f:
            f.write(f'{link}\n')
    except:
        with open('../resource/data/failed_to_get_csv_link.txt', 'a') as f:
            f.write(f'{link}\n')

Attach the name of each city to the link

In [12]:
## get list of cities
cities = browser.find_elements(By.XPATH, "//span[text()='Search for homes by city']/following-sibling::ul/child::li")

In [20]:
# connect to database
conn = sqlite3.connect('../resource/data/homes_for_sale_by_city.db')
cur = conn.cursor()

In [25]:
cur.execute('DROP TABLE IF EXISTS test')
cur.execute('CREATE TABLE test (id INTEGER PRIMARY KEY, \
                                city TEXT, \
                                link TEXT, \
                                UNIQUE(city, link))')

In [26]:
cur.execute('INSERT INTO test (city, link) VALUES (?, ?)', ('New York', 'abc'))

In [ ]:
# prevent duplicated row except the "id" column
cur.execute('INSERT INTO test (city, link) VALUES (?, ?)', ('New York', 'abc'))
cur.execute('INSERT INTO test (city, link) VALUES (?, ?)', ('New York', 'abc'))

In [12]:
# insert null or leave empty if null into table
##
cur.execute('INSERT INTO test (city, link) VALUES (?, ?)', ('Texas', None))
##
cur.execute('INSERT INTO test (city) VALUES (?)', ('Texas',))

In [24]:
cur.execute('SELECT * FROM test').fetchall()

[(1, 'New York', 'abc'), (2, 'New York', 'abc')]

In [28]:
cur.close()
conn.close()